# Hyperparameter Tuning

## Imports

In [1]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from triplet_utils import *
from utils import *
from torch.utils.data import random_split
from triplet_utils import train_and_evaluate_triplet


### Set up the model

In [2]:
# TripletModel definition
class TripletModel(nn.Module):
    def __init__(self):
        super(TripletModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 128)  # Adjust output size for embedding

    def forward(self, x):
        # Get the embedding directly from the base model
        return self.base_model(x)  # Call the model with input x

class TripletClassifier(nn.Module):
    def __init__(self, triplet_head, output_size=1):
        super(TripletClassifier, self).__init__()
        self.triplet_head = triplet_head
        self.classifier = nn.Linear(self.triplet_head.base_model.fc.out_features, output_size)  # Binary classification (logits)

    def forward(self, x):
        embedding = self.triplet_head(x)  # Output of base model
        class_output = self.classifier(embedding)  # Output for classification
        return embedding, class_output

### Set your dataset directory

In [3]:
# Load data
data_dir = '../../data/BiteCount/salient_poses/'

# Define transformations for training set and validation set
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
dataset = TripletDataset(data_dir, transform=train_transforms)
dataset_size = len(dataset)
class_names = dataset.classes

dataset_size = len(dataset)
class_names = dataset.classes

# # Cross-validation setup
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Split the dataset randomly into training and validation sets
train_size = int(0.8 * dataset_size)  # 80% training, 20% validation
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Hyperparameters
batch_size = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 25
patience = 5
alpha = 0.5

# Placeholder to store metrics across folds
val_accuracies = []
val_f1_scores = []
precisions = []
recalls = []
roc_aucs = []

### Set your model and parameters

In [4]:
def custom_collate(batch):
    # Unpack the batch
    anchors, positives, negatives, labels = zip(*batch)

    # Stack tensors to create batches
    anchors = torch.stack(anchors)
    positives = torch.stack(positives)
    negatives = torch.stack(negatives)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    return anchors, positives, negatives, labels

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

# Perform 5-Fold Cross Validation
fold_idx = 1
num_epochs = 25
patience = 5

## Tune the model

In [6]:
# # Tuning: Experiment with different values for learning rate and momentum
# alphas = [i*0.1 for i in range(1, 11)]

# best_val_acc = 0.0
# best_lr = None
# best_momentum = None
# best_val_loss = 1e10

# for alpha in alphas:
#     print(f"Training with alpha: {alpha}")
#     # Initialize TripletModel
#     triplet_model = TripletModel()
#     model = TripletClassifier(triplet_model)
#     model = model.to(device)

#     # Define loss functions and optimizer
#     criterion_classification = nn.BCEWithLogitsLoss()  # Binary cross-entropy with logits for binary classification
#     criterion_triplet = nn.TripletMarginLoss()  # Triplet margin loss
#     optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
#     patience_counter = 0

#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0

#         for batch in train_loader:
#             # Unpack the batch correctly
#             anchor, positive, negative, labels = batch
            
#             # Move the tensors to the appropriate device
#             anchor = anchor.to(device)
#             positive = positive.to(device)
#             negative = negative.to(device)
#             labels = labels.to(device)  # Ensure labels are tensors

#             # Forward pass
#             anchor_embedding, anchor_output = model(anchor)
#             positive_embedding, positive_output = model(positive)
#             negative_embedding, negative_output = model(negative)

#             # Calculate losses
#             classification_loss = criterion_classification(anchor_output, labels)
#             triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)

#             # Combine losses
#             loss = alpha * classification_loss + (1-alpha) * triplet_loss
#             running_loss += loss.item()

#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#         avg_loss = running_loss / len(train_loader)
#         print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

#         # Validation step
#         model.eval()
#         val_preds, val_labels = [], []
#         val_loss = 0.0

#         with torch.no_grad():
#             for batch in val_loader:
#                 anchor, positive, negative, labels = batch
                
#                 anchor = anchor.to(device)
#                 positive = positive.to(device)
#                 negative = negative.to(device)
#                 labels = labels.to(device)

#                 anchor_embedding, anchor_output = model(anchor)
#                 positive_embedding, positive_output = model(positive)
#                 negative_embedding, negative_output = model(negative)

#                 # Compute validation loss
#                 classification_loss = criterion_classification(anchor_output, labels)
#                 triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)
#                 val_loss += (classification_loss + triplet_loss).item()

#                 val_preds.append(torch.sigmoid(anchor_output).cpu().numpy())
#                 val_labels.append(labels.cpu().numpy())

#         avg_val_loss = val_loss / len(val_loader)
#         val_preds = np.concatenate(val_preds)
#         val_labels = np.concatenate(val_labels)

#         # Calculate metrics
#         val_acc = accuracy_score(val_labels, (val_preds > 0.5).astype(int))
#         val_f1 = f1_score(val_labels, (val_preds > 0.5).astype(int))
#         precision = precision_score(val_labels, (val_preds > 0.5).astype(int))
#         recall = recall_score(val_labels, (val_preds > 0.5).astype(int))
#         roc_auc = roc_auc_score(val_labels, val_preds)

#         # Print validation metrics
#         print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}, "
#               f"Precision: {precision:.4f}, Recall: {recall:.4f}, ROC AUC: {roc_auc:.4f}")

#         # Early stopping logic
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             best_alpha = alpha
#             patience_counter = 0
#             # Save model state
#             torch.save(model.state_dict(), f'triplet_classifier_fold_{fold_idx}.pth')
#         else:
#             patience_counter += 1
#             if patience_counter >= patience:
#                 print("Early stopping triggered.")
#                 break



In [7]:
# print(f"Best alpha: {best_alpha}.")

In [8]:
import itertools
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Define the range of hyperparameters to test
margins = [i * 0.1 for i in range(1, 11)]  # Margin from 0.1 to 1.0
alphas = [0.1, 0.3, 0.5, 0.7, 0.9]  # Different values for alpha
learning_rates = [0.001, 0.01, 0.1]  # Example learning rates
momentums = [0.5, 0.8, 0.9]  # Different momentum values

# Store the best results
best_val_acc = 0.0
best_hyperparams = {}
best_val_loss = 1e10

# Create a search space using itertools
search_space = itertools.product(margins, alphas, learning_rates, momentums)

for margin, alpha, lr, momentum in search_space:
    print(f"Training with margin: {margin}, alpha: {alpha}, learning_rate: {lr}, momentum: {momentum}")

    # Initialize TripletModel
    triplet_model = TripletModel()
    model = TripletClassifier(triplet_model)
    model = model.to(device)

    # Define loss functions and optimizer
    criterion_classification = nn.BCEWithLogitsLoss()  # Binary cross-entropy for classification
    criterion_triplet = nn.TripletMarginLoss(margin=margin)  # Use margin from the search space
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # Optimizer with dynamic lr and momentum
    patience_counter = 0

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch in train_loader:
            anchor, positive, negative, labels = batch
            anchor, positive, negative, labels = anchor.to(device), positive.to(device), negative.to(device), labels.to(device)

            # Forward pass
            anchor_embedding, anchor_output = model(anchor)
            positive_embedding, positive_output = model(positive)
            negative_embedding, negative_output = model(negative)

            # Calculate losses
            classification_loss = criterion_classification(anchor_output, labels)
            triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)

            # Combine losses
            loss = alpha * classification_loss + (1 - alpha) * triplet_loss
            running_loss += loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

        # Validation step
        model.eval()
        val_preds, val_labels = [], []
        val_loss = 0.0

        with torch.no_grad():
            for batch in val_loader:
                anchor, positive, negative, labels = batch
                anchor, positive, negative, labels = anchor.to(device), positive.to(device), negative.to(device), labels.to(device)

                anchor_embedding, anchor_output = model(anchor)
                positive_embedding, positive_output = model(positive)
                negative_embedding, negative_output = model(negative)

                # Compute validation loss
                classification_loss = criterion_classification(anchor_output, labels)
                triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)
                val_loss += (classification_loss + triplet_loss).item()

                val_preds.append(torch.sigmoid(anchor_output).cpu().numpy())
                val_labels.append(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_preds = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)

        # Calculate validation metrics
        val_acc = accuracy_score(val_labels, (val_preds > 0.5).astype(int))
        val_f1 = f1_score(val_labels, (val_preds > 0.5).astype(int))
        precision = precision_score(val_labels, (val_preds > 0.5).astype(int))
        recall = recall_score(val_labels, (val_preds > 0.5).astype(int))
        roc_auc = roc_auc_score(val_labels, val_preds)

        print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}, "
              f"Precision: {precision:.4f}, Recall: {recall:.4f}, ROC AUC: {roc_auc:.4f}")

        # Early stopping and best model saving
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_val_acc = val_acc
            best_hyperparams = {'margin': margin, 'alpha': alpha, 'learning_rate': lr, 'momentum': momentum}
            patience_counter = 0
            torch.save(model.state_dict(), f'triplet_classifier_best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

# Save the results to a text file
with open('best_hyperparameters_tmm.txt', 'w') as f:
    f.write(f"Best validation loss: {best_val_loss:.4f}\n")
    f.write(f"Best accuracy: {best_val_acc:.4f}\n")
    f.write(f"Best hyperparameters:\n")
    f.write(f"  Margin: {best_hyperparams['margin']}\n")
    f.write(f"  Alpha: {best_hyperparams['alpha']}\n")
    f.write(f"  Learning Rate: {best_hyperparams['learning_rate']}\n")
    f.write(f"  Momentum: {best_hyperparams['momentum']}\n")

print("Results saved to 'best_hyperparameters_results.txt'")


Training with margin: 0.1, alpha: 0.1, learning_rate: 0.001, momentum: 0.5


/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/25], Loss: 1.4759
Validation Loss: 2.1951, Accuracy: 0.4750, F1 Score: 0.4474, Precision: 0.4722, Recall: 0.4250, ROC AUC: 0.4806
Epoch [2/25], Loss: 1.2775
Validation Loss: 2.0142, Accuracy: 0.5375, F1 Score: 0.4789, Precision: 0.5484, Recall: 0.4250, ROC AUC: 0.5062
Epoch [3/25], Loss: 1.1008
Validation Loss: 1.7801, Accuracy: 0.5250, F1 Score: 0.5250, Precision: 0.5250, Recall: 0.5250, ROC AUC: 0.4875
Epoch [4/25], Loss: 0.9404
Validation Loss: 1.7411, Accuracy: 0.4750, F1 Score: 0.4615, Precision: 0.4737, Recall: 0.4500, ROC AUC: 0.4875
Epoch [5/25], Loss: 0.9179
Validation Loss: 1.5866, Accuracy: 0.4875, F1 Score: 0.4533, Precision: 0.4857, Recall: 0.4250, ROC AUC: 0.5119
Epoch [6/25], Loss: 0.8086
Validation Loss: 1.5542, Accuracy: 0.5000, F1 Score: 0.5122, Precision: 0.5000, Recall: 0.5250, ROC AUC: 0.4931
Epoch [7/25], Loss: 0.7983
Validation Loss: 1.5738, Accuracy: 0.4625, F1 Score: 0.4267, Precision: 0.4571, Recall: 0.4000, ROC AUC: 0.4594
Epoch [8/25], Loss: 0.7257


/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/25], Loss: 1.4703
Validation Loss: 2.2280, Accuracy: 0.3750, F1 Score: 0.3243, Precision: 0.3529, Recall: 0.3000, ROC AUC: 0.3569
Epoch [2/25], Loss: 1.1818
Validation Loss: 1.8577, Accuracy: 0.4000, F1 Score: 0.2500, Precision: 0.3333, Recall: 0.2000, ROC AUC: 0.4038


KeyboardInterrupt: 